In [1]:
import sys
sys.path.append('../src')

import os
import requests
import json
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta

from betfair import Betfair

pd.set_option('display.max_columns', None)

In [2]:
# need to import daily data so that we can get the race ids and loop through for results.

In [3]:
app_key = 'ABGJLOlKaLtTsMIp'

username1 = input("Email:")
password1 = input("Password:")

In [4]:
trading = betfairlightweight.APIClient(
        username1, password1, app_key
    )
trading.login_interactive()

trading.session_token

trading.keep_alive()

trading.race_card.login()

In [5]:
folder_path = '../data/daily'

csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

In [8]:
def fetch_race_results(market_ids):
    try:
        data = trading.race_card.get_race_result(market_ids=market_ids)
    except Exception as e:
        print(f"An error occurred: {e}")
        data = []

    # Process and display the data
    flat_data = []

    for race in data:
        for runner in race.get('runners', []):
            for selection in runner.get('selections', []):
                if selection['marketType'] == 'WIN' and 'bsp' in selection:
                    flat_data.append({
                        'race_id': race.get('raceId'),
                        'start_date': race.get('startDate'),
                        'course_name': race.get('course', {}).get('name'),
                        'country_code': race.get('course', {}).get('countryCode'),
                        'race_title': race.get('raceTitle'),
                        'race_class': race.get('raceClassification', {}).get('classification'),
                        'distance': race.get('distance'),
                        'course_type': race.get('course', {}).get('courseType'),
                        'surface_type': race.get('course', {}).get('surfaceType'),
                        'market_id': selection.get('marketId'),
                        'market_type': selection.get('marketType'),
                        'horse_id': runner.get('horseId'),
                        'horse_name': runner.get('horseName', 'Unknown'),
                        'saddle_cloth': runner.get('saddleCloth'),
                        'isNonRunner': runner.get('isNonRunner'),
                        'position': runner.get('position'),
                        'selectionId': selection.get('selectionId'),
                        'bsp': selection.get('bsp')
                    })

    return pd.DataFrame(flat_data)

In [9]:
merged_dfs = []

for csv in csv_files:

    df = pd.read_csv(os.path.join(folder_path, csv))
    df['market_id'] = df['market_id'].astype(str)
    df['market_id'] = df['selection_id'].astype(str)

    market_ids = df['market_id'].unique()

    results_df = fetch_race_results(market_ids)

    results_df['market_id'] = results_df['market_id'].astype(str)
    results_df['selection_id'] = results_df['selection_id'].astype(str)

    df_merged = df.merge(results_df, left_on = ['market_id', 'selection_id'], right_on= ['market_id', 'selection_id'],how = 'left')

    merged_dfs.append(df_merged)

final_df = pd.concat(merged_dfs, ignore_index=True)

final_df.to_csv('../data/results/results.csv', index=False)

An error occurred: Status code error: 400


KeyError: 'market_id'

In [ ]:
market_ids

array([1.23204517, 1.23204511, 1.23204574, 1.23204575, 1.23204499,
       1.23204577, 1.232045  , 1.23204579, 1.232045  , 1.2320458 ,
       1.23204501, 1.2320458 , 1.23204501, 1.23204581, 1.23204502])

In [ ]:
market_ids = ["1.231693932"]

try:
    data = trading.race_card.get_race_result(market_ids=market_ids)
except Exception as e:
    print(f"An error occurred: {e}")
    data = []

# Process and display the data
flat_data = []

for race in data:
    for runner in race.get('runners', []):
        for selection in runner.get('selections', []):
            if selection['marketType'] == 'WIN' and 'bsp' in selection:
                flat_data.append({
                    'raceId': race.get('raceId'),
                    'startDate': race.get('startDate'),
                    'courseName': race.get('course', {}).get('name'),
                    'countryCode': race.get('course', {}).get('countryCode'),
                    'raceTitle': race.get('raceTitle'),
                    'raceClass': race.get('raceClassification', {}).get('classification'),
                    'raceDistance': race.get('distance'),
                    'courseType': race.get('course', {}).get('courseType'),
                    'surfaceType': race.get('course', {}).get('surfaceType'),
                    'marketId': selection.get('marketId'),
                    'marketType': selection.get('marketType'),
                    'horseId': runner.get('horseId'),
                    'horseName': runner.get('horseName', 'Unknown'),
                    'saddleCloth': runner.get('saddleCloth'),
                    'isNonRunner': runner.get('isNonRunner'),
                    'position': runner.get('position'),
                    'selectionId': selection.get('selectionId'),
                    'bsp': selection.get('bsp')
                })

# Create a DataFrame
import pandas as pd
df = pd.DataFrame(flat_data)

# Display the DataFrame
print(df)

In [ ]:
# need to save down the results
